# Dudas y preguntas frecuentes

### Las ventanas no muestran las imágenes

Hace falta `cv.waitKey(ms)` para [refrescar el interfaz](https://docs.opencv.org/4.1.0/d7/dfc/group__highgui.html#ga5628525ad33f52eab17feebcfba38bd7) de opencv. Esto se hace automáticamente dentro del bucle de captura de autoStream(). En cualquier otro caso es necesario llamar a esta función para que se refresquen la ventanas aunque no necesitemos la tecla pulsada.

### Un módulo no tiene las funciones que debería tener

Si un archivo del directorio de trabajo tiene el mismo nombre que un módulo del sistema (por ejemplo "numpy.py", "dlib.py", etc.), un import de ese módulo producirá un error: Python cargará el archivo local en lugar del módulo y no encontrará sus funciones. Simplemente hay que tener cuidado con los nombres de archivo de nuestro código fuente para que no coincidan con ningún módulo.

### Ejercicio COLOR

Se trata de crear una aplicación parecida a la que se muestra en el siguiente pantallazo: La ventana superior es la imagen en vivo de la webcam (o la fuente de video deseada). Cuando se marca un ROI con el ratón se muestran los histogramas (normalizados) de los 3 canales por separado. Y si se pulsa una cierta tecla se guarda el recuadro como un modelo más y se muestra en la ventana "models" de abajo a la izquierda. En este caso vemos que se han guardado ya tres modelos. En todo momento (siempre que haya algún modelo guardado) se comparan los histogramas del ROI actual con los de todos los modelos. Las distancias se muestran arriba a la izquierda. La menor, 0.32, nos indica que el segundo modelo es el más parecido, y se muestra en la ventana "detected". Si la menor distancia es muy grande se puede rechazar la decisión y y mostrar un recuadro negro.

La comparación entre histogramas puede hacerse de muchas formas. Una muy sencilla es la suma de diferencias absolutas en cada canal y quedarnos el máximo de los tres canales.

Los modelos serán en general rectángulos de tamaños diferentes, y que tampoco tienen por qué coincidir con el tamaño del ROI que queremos clasificar. Esto implica que los histogramas deben normalizarse.

La ventana de models puede construirse con unas miniaturas reescaladas a una misma altura predeterminada y ancho proporcional al original, o simplemente a un cuadrado fijo.

![HISTCOL](../images/demos/HISTCOL.png)

### Ejercicio SIFT

Algunas recomendaciones:

Los objetos deben tener "detalles" para que aparezcan dentro suficientes puntos de interés. Hay objetos muy uniformes que producen muy pocos puntos, o salen casi todos por los bordes o el exterior, y entonces no son muy apropiados para este método. Por ejemplo una libreta con tapas negras, o un teléfono móvil, una botella de cristal sin etiqueta, etc. no funcionan bien.

Pulsando una tecla se pueden ir guardando modelos sobre la marcha, pero es conveniente leerlos de una carpeta para agilizar el uso del programa.

Hay que calcular una sola vez los keypoints y descriptores de los modelos. En el bucle de captura solo deben calcularse los de la imagen actual.

Para reducir el tiempo de cálculo se puede trabajar con imágenes me menor resolución (eg. 400x300) y limitar el número de keypoints en las comparaciones (parámetro nfeatures).

Usando autoStream() lo normal es que se produzca un retardo en las imágenes, por lo que es preferible usar la captura con hilo mediante la utilidad Camera de umucv. 

Sí a pesar de todo el proceso va muy lento, se puede efectuar la extracción de keypoints y la comparación con los modelos solo cuando detectemos que la imagen está bastante quieta o cuando pulsemos una tecla.

Si los modelos tienen diferente número de keypoints la comparación debe hacerse teniendo en cuenta el porcentaje de coincidencias, no el valor absoluto.

Se puede rechazar la decisión cuando el porcentaje ganador sea pequeño, cuando el segundo mejor sea parecido al primero, o cuando haya pocas coincidencias en la imagen, entre otras situaciones que dependen de la aplicación.

Cuando no se detecta ningún keypoint los descriptores no se devuelven como un array de dimension 0x128, sino como un valor `None`. Hay que tener cuidado con  esto para que no se produzcan errores en tiempo de ejecución. Esto puede ocurrir cuando la cámara apunta hacia la mesa, o está muy desenfocada.

Aquí hay un [vídeo de ejemplo](https://robot.inf.um.es/material/va/sift-demo.mp4) de lo que se puede conseguir sin muchas complicaciones. Se muestra en pequeño el modelo ganador, su porcentaje, y la diferencia con el segundo mejor. Observa que los objetos se reconocen aunque no se vean enteros en la imagen, con diferentes tamaños, con cualquier rotación en el plano de imagen, y con cierta inclinación de perspectiva. Hay también cierta resistencia al desenfoque y a reflejos. (Aunque no se ven en esta breve secuencia, pueden producirse clasificaciones erróneas cuando la escena tiene muchos puntos y no hay ningún modelo conocido.)

Como alternativa a SIFT también se puede utilizar el método AKAZE, que funciona igual de bien o mejor. Solo hay que cambiar una línea:

    # sift = cv.SIFT_create( ..parámetros.. )
    sift = cv.AKAZE_create()   # tiene otros parámetros pero la configuración por omisión funciona bastante bien.

### Ejercicio VROT 

Inicialmente vamos a estudiar solo los giros en los ángulos "[pan][pan]" (izquierda-derecha) y "[tilt][tilt]" (arriba-abajo).

[pan]:https://en.wikipedia.org/wiki/Panning_(camera)

[tilt]: https://en.wikipedia.org/wiki/Tilt_(camera)

En este ejercicio lo más sencillo es partir de code/lk_tracks.py y reducir la longitud de los tracks para que sus longitudes se aproximen más al movimiento instantáneo. El siguiente pantallazo muestra el efecto de un giro de la cámara hacia la izquiera y ligeramente hacia abajo. Como la escena es estática, el movimiento de todos los puntos es bastante regular. (No es exactamente igual de largo porque sabemos que la relación de ángulos y pixels lleva una arcotangente, pero si el FOV no es muy grande las diferencias no son importantes).

En naranja dibujamos el vector medio de todos esos desplazamientos. Lo dibujamos desde el centro de la imagen, ampliado para que vea mejor y en dirección opuesta (la del giro de la cámara).

Con esto tenemos el desplazamiento de la cámara en los últimos $n$ frames (la longitud de los tracks), que podemos pasar a pixels/frame. Con los parámetros de la cámara obtenidos en el ejercicio FOV podemos convertirlo a grados/frame. Y con el período de muestreo, que podemos medir con time.time() o deducirlo de los fps de captura, podemos pasarlo a grados/segundo.

![vrot](../images/demos/vrot.png)

Opcionalmente, además de la velocidad de giro, puedes mostrar en pantalla el ángulo total de rotación acumulado en cada eje, para comprobar a ojo que el valor es aproximadamente correcto cuando giras la cámara un ángulo conocido de p. ej. 90 grados.

En este ejercicio no buscamos mucha precisión. Lo importante es obtener el orden de magnitud correcto de las medidas.